In [ ]:
#mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip '/content/drive/MyDrive/Untitled folder/FER2013/FER2013.zip' -d '/content/drive/MyDrive/Untitled folder/FER2013'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65267116.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65275626.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_6529266.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65329617.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65338712.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65338797.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65387162.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65404494.jpg  
  inflating: /content/drive/MyDrive/Untitled folder/FER2013/FER2013/train/sad/Training_65426218.jpg  
  inflating: /content/drive/MyDr

In [ ]:
#Import Required Library
import cv2
import os
#define Preprocessing function
def preprocess_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (48, 48))
    cv2.imwrite(output_path, img)

# Replace with the path to your root directory containing the emotion folders
root_dir = '/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Train'

# Replace with the desired output directory
output_dir = '/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Preporcessed'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate through each
emotion_folder = os.listdir(root_dir)
for emotion_folder in os.listdir(root_dir):
    emotion_path = os.path.join(root_dir, emotion_folder)
    output_emotion_path = os.path.join(output_dir, emotion_folder)

    if not os.path.exists(output_emotion_path):
        os.makedirs(output_emotion_path)

    for image_file in os.listdir(emotion_path):
        image_path = os.path.join(emotion_path, image_file)
        output_file = os.path.join(output_emotion_path, image_file)
        preprocess_image(image_path, output_file)
#Print process
print("Image preprocessing complete!")

Image preprocessing complete!


In [ ]:
#import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
#import datasets
dataset1_path = '/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Preporcessed'
dataset2_path = '/content/drive/MyDrive/Untitled folder/CK+48/CK+48'

# Define the image dimensions
img_width, img_height = 48,48

# Define the batch size
batch_size = 32

# Create ImageDataGenerator instances for both datasets
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator1 = datagen.flow_from_directory(
    dataset1_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

train_generator2 = datagen.flow_from_directory(
    dataset2_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 52 images belonging to 7 classes.
Found 981 images belonging to 7 classes.


In [ ]:
# Create combined genarator
def combined_generator(gen1, gen2):
    while True:
        try:
            x1, y1 = next(gen1)  # Try to get data from generator1
        except StopIteration:
            x1, y1 = None, None  # If generator1 is exhausted, set to None
        try:
            x2, y2 = next(gen2)  # Try to get data from generator2
        except StopIteration:
            break  # If generator2 is exhausted, break the loop

        if x1 is not None and x2 is not None:  # If both generators have data
            yield np.concatenate([x1, x2]), np.concatenate([y1, y2])

combined_generator = combined_generator(train_generator1, train_generator2)



In [ ]:
#Import Required Library
import numpy as np
from tensorflow.keras.models import Model

In [ ]:
# Load the pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
from tensorflow.keras.layers import Dropout
# Freeze the base model layers
for layer in base_model.layers[-10:]:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax')(x)  # Adjusting the number of classes as per emotions

# Create the final model
model1 = Model(inputs=base_model.input, outputs=x)

# Compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
for _ in range(len(train_generator1) + len(train_generator2)):
    next(combined_generator)  # Consume data until exhaustion
# Train the model
model1.fit(combined_generator,
          steps_per_epoch=max(len(train_generator1), len(train_generator2)),
          epochs=5,
          validation_data=combined_generator,
          validation_steps=max(len(train_generator1), len(train_generator2)))

Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 288s 8s/step - accuracy: 0.4399 - loss: 1.7675 - val_accuracy: 0.1991 - val_loss: 6.5623
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 244s 8s/step - accuracy: 0.8015 - loss: 0.5445 - val_accuracy: 0.1280 - val_loss: 3.0904
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 244s 8s/step - accuracy: 0.8861 - loss: 0.3758 - val_accuracy: 0.1249 - val_loss: 3.5926
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 245s 8s/step - accuracy: 0.9385 - loss: 0.2042 - val_accuracy: 0.1283 - val_loss: 4.3910
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 244s 8s/step - accuracy: 0.9024 - loss: 0.3404 - val_accuracy: 0.1233 - val_loss: 4.0090


In [ ]:
# Save the model to a file
model1.save('Model2Custom_CK.h5')
# Save the model to a file
model1.save('Model2Custom_CK.keras')  # Save as a single HDF5 file  # Save as a single HDF5 file

In [ ]:
#import datasets
dataset1_path = '/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Preporcessed'
dataset3_path = '/content/drive/MyDrive/Untitled folder/FER2013/FER2013/train'

# Define the image dimensions
img_width, img_height = 48,48

# Define the batch size
batch_size = 32

# Create ImageDataGenerator instances for both datasets
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator1 = datagen.flow_from_directory(
    dataset1_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

train_generator3 = datagen.flow_from_directory(
    dataset3_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 52 images belonging to 7 classes.
Found 28709 images belonging to 7 classes.


In [ ]:
#create combined generator
def combined_generator(gen1, gen2):
    while True:
        try:
            x1, y1 = next(gen1)  # Try to get data from generator1
        except StopIteration:
            x1, y1 = None, None  # If generator1 is exhausted, set to None
        try:
            x2, y2 = next(gen2)  # Try to get data from generator2
        except StopIteration:
            break  # If generator2 is exhausted, break the loop

        if x1 is not None and x2 is not None:  # If both generators have data
            yield np.concatenate([x1, x2]), np.concatenate([y1, y2])

combined_generator = combined_generator(train_generator1, train_generator2)

#import library
import numpy as np
from tensorflow.keras.models import Model



In [ ]:
# Load the pre-trained ResNet50 model, excluding the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
from tensorflow.keras.layers import Dropout
# Freeze 10 the base model layers
for layer in base_model.layers[-10:]:
    layer.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax')(x)  # Adjusting the number of classes as per emotions

# Create the final model
model4 = Model(inputs=base_model.input, outputs=x)

# Compile the model
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
for _ in range(len(train_generator1) + len(train_generator2)):
    next(combined_generator)  # Consume data until exhaustion
# Train the model
model4.fit(combined_generator,
          steps_per_epoch=max(len(train_generator1), len(train_generator2)),
          epochs=5,
          validation_data=combined_generator,
          validation_steps=max(len(train_generator1), len(train_generator2)))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 299s 8s/step - accuracy: 0.3940 - loss: 1.8422 - val_accuracy: 0.1016 - val_loss: 32.1620
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 254s 8s/step - accuracy: 0.7757 - loss: 0.6585 - val_accuracy: 0.1825 - val_loss: 3.3935
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 249s 8s/step - accuracy: 0.8596 - loss: 0.4700 - val_accuracy: 0.1039 - val_loss: 5.2001
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 260s 8s/step - accuracy: 0.8997 - loss: 0.3117 - val_accuracy: 0.1325 - val_loss: 5.6645
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 252s 8s/step - accuracy: 0.9327 - loss: 0.2611 - val_accuracy: 0.0882 - val_loss: 6.0425


In [ ]:
# Save the model to a file
model4.save('Model4Custom_CK.h5')
# Save the model to a file
model4.save('Model4Custom_CK.keras')  # Save as a single HDF5 file  # Save as a single HDF5 file

In [ ]:
#Import Required Library
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelEncoder


# Function to load and preprocess the image
def load_and_preprocess_image(img_path, target_size=(img_width, img_height)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Path to the image to predict
img_path = ('/content/drive/MyDrive/Untitled folder/Custom dataset/Face/Preporcessed/sad/Ankita_3.png')
# Load and preprocess the image
new_img = load_and_preprocess_image(img_path)

# Make prediction
predictions = model4.predict(new_img)
print(predictions)
# Get the class index with the highest probability
predicted_class_index = np.argmax(predictions, axis=1)

# Convert the class index to the original class label
predicted_emotion = le.inverse_transform(predicted_class_index)[0]  # Get the label

# Print the predicted emotion
print("Predicted emotion:", predicted_emotion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
[[1.8189429e-09 1.6319253e-04 1.5502599e-07 4.4061709e-03 3.8423497e-07
  1.1678122e-10 9.9543011e-01]]
Predicted emotion: happy


In [ ]:
model4.save_weights('model4.weights.h5')